# Functional Comparison of ASN with WS and C. Elegans

This notebook generates 24 different ASN networks with 300 nodes, each of which has differing parameters (Average Path length, Wire Dispersion and Centroid Dispersion). 

We then find the average degree for each ASN network, and use that as 2k to generate corresponding grid-like, small-world and random Watts-Strogatz networks.

We also load a sample C. Elegans network for comparison.

In [1]:
# !jupyter notebook --version
# !python --version
# !conda --version
# # !jupyter trust RunTasksDifferentNetworks_MultipleNetworks.ipynb

In [2]:
cd

/suphys/aloe8475


In [3]:
cd "Documents/edamame"

/import/silo2/aloe8475/Documents/edamame


In [4]:
from scipy.io import loadmat, savemat
from scipy.stats import kurtosis

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime
import networkx as nx
from edamame import *
from tqdm import tqdm_notebook
import os
import edamame.core.wires as wires
from random import choice
import warnings
from IPython.core.debugger import set_trace
import nct
import bct

#warnings.filterwarnings('ignore')

## Functions:

In [5]:
import pickle 
import _pickle as cPickle
import gzip
def compressed_pickle(obj, filename,protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)

In [6]:
def decompress_pickle(file):
    with gzip.open(file, 'rb') as f:
        loaded_object = cPickle.load(f)
        return loaded_object

In [7]:
# this_seed2=700
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

In [8]:
#Select Largest Components
def select_largest_component_new(wires_dict):
    """
    Find and select largest connected component of the original graph G.
    Throws away unconnected components and updates all the keys in wires_dict 
    """
#     def connected_component_subgraphs(G):
#         for c in nx.connected_components(G):
#             yield G.subgraph(c)
    
    wires_dict['G'] = max(connected_component_subgraphs(wires_dict['G']), key=len)
#     set_trace()
    nw = len(wires_dict['G'].nodes())
    nj = len(wires_dict['G'].edges())   
    
    logging.info("The largest component has %5d nodes and %6d edges", nw, nj)

    # Replace values in the dictionary
    wires_dict['number_of_wires']     = nw
    wires_dict['number_of_junctions'] = nj
    wires_dict['xa'] = wires_dict['xa'][wires_dict['G'].nodes()] 
    wires_dict['ya'] = wires_dict['ya'][wires_dict['G'].nodes()] 
    wires_dict['xb'] = wires_dict['xb'][wires_dict['G'].nodes()] 
    wires_dict['yb'] = wires_dict['yb'][wires_dict['G'].nodes()]
    wires_dict['xc'] = wires_dict['xc'][wires_dict['G'].nodes()] 
    wires_dict['yc'] = wires_dict['yc'][wires_dict['G'].nodes()]
 
    # Keep old edge_list
    old_edge_list = [(ii, kk) for ii, kk in  zip(wires_dict['edge_list'][:, 0], wires_dict['edge_list'][:, 1])]
    # Remove old edge list
    wires_dict = wires.remove_key(wires_dict, 'edge_list') 
    # Save indices of intersections in the old graph
    ind_dict = {key:value for value,key in enumerate(old_edge_list)}
    new_edge_list = sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0])
    # Find intersection between the two sets
    inter = set(ind_dict).intersection(new_edge_list)
    # Retrieve edge indices/positions from the old list
    edges_idx = [ind_dict[idx] for idx in inter]
       
    # These have length equal to number of junctions -- only get the ones we need
    wires_dict['xi'] = wires_dict['xi'][edges_idx] 
    wires_dict['yi'] = wires_dict['yi'][edges_idx] 
    
    # Get contiguous numbering of nodes
    # Build node mapping 
    node_mapping    = {key:value for value, key in enumerate(sorted(wires_dict['G'].nodes()))}
    # This  step also renames edges list
    wires_dict['G'] =  nx.relabel_nodes(wires_dict['G'] , node_mapping)

    # Swap node vertices if vertex 0 is larger than vertex 1, then sort by first element
    wires_dict['edge_list'] = np.asarray(sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0]))
    
    # Save adjacency matrix of new graph
    wires_dict = wires.remove_key(wires_dict, 'adj_matrix') 
    wires_dict = wires.generate_adj_matrix(wires_dict)

    wire_distances = wires.cdist(np.array([wires_dict['xc'], wires_dict['yc']]).T, np.array([wires_dict['xc'], wires_dict['yc']]).T, metric='euclidean')    
    wires_dict['wire_distances'] = wire_distances

    return wires_dict 

In [9]:
def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=1.)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

## Load Structural Connectivity

In [11]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


### Elegans

In [12]:
name='elegans_LinearTransformation.pkl'
print('Loading Elegans Networks')
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[Elegans] = pickle.load(file)

print('Loaded')

Loading Elegans Networks
Loaded


In [40]:
elegansGraph=Elegans['G']

In [13]:
#Small world calculated on C Elegans Matrix in smallworld.m in MATLAB
temp=loadmat(r'cElegans_smallworld.mat')
smallworld_elegans=temp['cElegansSW'][0]
del temp

### ASNs

In [14]:
# Small Worldness: 
# ------------------------------------
# CALCULATED IN MATLAB: smallworldness.m 
# found in C:\Users\aloe8475\Documents\PhD\GitHub\CODE\Analysis\Functional Connectivity\Functional Tasks
# ------------------------------------
temp=loadmat(r'300nwASN_multipleNetworks_smallworld.mat')
smallworld=temp['smallworld']
del temp

In [15]:
name='networks_LinearTransformation.pkl'
print('Loading Networks + Linear Transformation Results')
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[ASN300] = pickle.load(file)

print('Loaded')

Loading Networks + Linear Transformation Results
Loaded


# Task 1: Linear  Transformation

In [22]:
#Regression
def NOKEVregression(target,absV): 
    inputx=np.vstack((np.ones(len(target)),absV)).T
    a1=np.linalg.lstsq(inputx,target)
    return a1

In [23]:
#Subgraph AdjMat

#Threshold by conductance - when tunnelling becomes appreciable (offResistance * 10)

def getOnGraph(network, this_TimeStamp = 0):
    edgeList = network.connectivity.edge_list
    adjMat = np.zeros((network.numOfWires, network.numOfWires))
#     set_trace()
    adjMat[edgeList[:,0], edgeList[:,1]] = (1/network.junctionResistance[this_TimeStamp,:])>1e-06#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
    adjMat[edgeList[:,1], edgeList[:,0]] = (1/network.junctionResistance[this_TimeStamp,:])>1e-06#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
    onGraph = nx.from_numpy_array(adjMat)
    onGraph=nx.DiGraph.to_undirected(onGraph)
    
    return onGraph

In [24]:
def getSubGraphComm(network, this_TimeStamp = 0):
    onGraph = getOnGraph(network, this_TimeStamp)
    components = [i for i in nx.connected_components(onGraph)]
    giant_component = components[np.argmax([len(i) for i in nx.connected_components(onGraph)])]
    nodes = list(giant_component)
    commMat = np.zeros((network.numOfWires, network.numOfWires))
    subComm = nx.communicability(onGraph.subgraph(giant_component))
    for i in nodes:
        for j in nodes:
            commMat[i,j] = subComm[i][j]
    return commMat

In [25]:
#Communicability + Current Matrices
def commCurr(sim):
    startTime=500
    timeSteps=50
    endTime=1500
    time_index=[startTime,endTime,timeSteps]#]len(sim.junctionResistance),timeSteps]
    currMat=[None]*len(range(startTime,endTime,timeSteps))#startTime,len(sim.junctionResistance),timeSteps))
    nodesListFull=[None]*len(range(startTime,endTime,timeSteps))#startTime,len(sim.junctionResistance),timeSteps))
    commu_Mat=[None]*len(range(startTime,endTime,timeSteps))#startTime,len(sim.junctionResistance),timeSteps))
    new_currGraph=[None]*len(range(startTime,endTime,timeSteps))#startTime,len(sim.junctionResistance),timeSteps))
    count = 0
    for i in tqdm(range(startTime,endTime,timeSteps)):#startTime,len(sim.junctionResistance),timeSteps)): #for each timestep
        currMat[count] = np.zeros((sim.numOfWires,sim.numOfWires))
        edgeList = sim.connectivity.edge_list
        currMat[count][edgeList[:,0], edgeList[:,1]] = sim.junctionVoltage[i,:]/sim.junctionResistance[i,:] #-1,:
        currMat[count] = currMat[count] + currMat[count].T
        currGraph = nx.from_numpy_array(currMat[count])
        subGraph = getOnGraph(sim, this_TimeStamp=i)
        commu_Mat[count]=getSubGraphComm(sim, this_TimeStamp=i)
        
        components = [j for j in nx.connected_components(subGraph)] #all connected nodes in subgraph

        max_ind = np.argmax([len(j) for j in nx.connected_components(subGraph)])
        currGraph = nx.subgraph(currGraph, components[max_ind])
        new_currGraph[count] = currGraph
        currMat[count] = np.array(nx.adjacency_matrix(new_currGraph[count]).todense())

#         commu = nx.communicability(new_currGraph[count])
#         commu_Edges[count]=commu
#         subSize = len(currGraph)
        nodesList=list(currGraph.nodes)
        nodesListFull[count]=nodesList

#         commu_Mat[count] = np.array([commu[k][j] for k in nodesList for j in nodesList]).reshape(subSize,subSize)
        count = count+1
        
    return nodesListFull,commu_Mat, currMat, new_currGraph, time_index

## ASN Networks:

In [26]:
onAmp=[[] for i in range(len(ASN300))]
shortestPath=[[None]*10 for i in range(len(ASN300))]
for i in tqdm(range(len(ASN300))):
    for j in range(len(ASN300[i])):
        temp=getFarthestPairing(ASN300[i][j]['adj_matrix'])
        shortestPath[i][j]=nx.shortest_path_length(ASN300[i][j]['G'],temp[0],temp[1])
        onAmp[i].append(shortestPath[i][j]/5) #need to justify divide by 5 or come up with some other way
        ASN300[i][j]['sp_amp']=onAmp[i][j]
        ASN300[i][j]['Graph Theory']['Shortest Path']=shortestPath[i][j]

In [11]:
def nonlineartrans(ASN300,onAmp,i,j,nodesList,stimulus):
    dt = 1e-2
    f=0.5
    Time=5
    period=1/f
   #Instantiate Variables    
    print('Running Linear Transformation Simulations')
    dt = 1e-2
    f=0.5
    Time=5

    #Choose Electrode Pattern
    stimulus[0][i].append((stimulus__(biasType='AC',onAmp=onAmp[i][j],TimeVector=np.arange(0,Time,dt),f=f)))
    stimulus[1][i].append((stimulus__(biasType='Drain',TimeVector=np.arange(0,Time,dt)))) #we don't want this drain to be active during training

    #Initialise Output Variables
    period=[]
    TimeVector=[]
    voltage=[]
    conductance=[]
    switches=[]
    #Run Simulations
    print('Parameter ' + str(i+1), ', Network ' + str(j+1))
    #Run Simulations
#     results=[]
    # Connectivity=connectivity__('700nw_14533junctions.mat')
    stimulus2 = [item for item in stimulus] #go through each list in the list and find the ith item
#     set_trace()
    results_ASN[j]=runSim(connectivity__(wires_dict=ASN300[i][j]),stimulus=stimulus2, contactMode='farthest', T = Time, dt = 0.001, onAmp = onAmp[i][j], biasType='AC',f=f,junctionMode='tunneling')
    #wires_dict=newNetworkTest[chosenNetwork])
    results_ASN[j].frequency=f
    results_ASN[j].dt=0.001
    period=1/f  

#         name= r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/simulations_LinearTransformation_' + str(i)
#         print('Saving 10 Simulations for Parameter ' +str(i+1))
#         compressed_pickle([results_ASN],name)

    print('Regressing Parameter ' + str(i+1), ', Network ' + str(j+1))
    TimeVector=results_ASN[j].TimeVector
    voltage=results_ASN[j].wireVoltage
    conductance=results_ASN[j].conductance
    switches=results_ASN[j].junctionSwitch

    target1= (onAmp[i][j] * (-np.sign(TimeVector % period - period/2)))
#             target2[j] = (4*onAmp[i]/period * abs((TimeVector-period/4) % period - period/2) - onAmp[i])
#             target3[j] = (onAmp[i]/period * (TimeVector % period))
#             target4[j] = (onAmp[i]*np.sin(4*np.pi*(1/period)*TimeVector))

    if len(ASN300[i][j]['G']) >= 250:
        nodesList[i][j]=[50,100,150,200,250,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)
    elif len(ASN300[i][j]['G']) >= 200 and len(ASN300[i][j]['G']) < 250:
        nodesList[i][j]=[50,100,150,200,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)  
    elif len(ASN300[i][j]['G']) >= 150 and len(ASN300[i][j]['G']) < 200:
        nodesList[i][j]=[50,100,150,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)  
    elif len(ASN300[i][j]['G']) >= 100 and len(ASN300[i][j]['G']) < 150:
        nodesList[i][j]=[50,100,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)
    
    nwSqu =[None]*len(nodesList[i][j])
    countK=0
    for k in nodesList[i][j]: #loop through sets of nodes for regression
        print('Running Regression: ' + str(k) + ' nodes')
        ResultSqu=[]
        ResultSqu = nonLinearTrans(results_ASN[j],'Square',k, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
        nwSqu[countK]=ResultSqu['accuracy']
        ASN300[i][j]['Accuracy']['Linear Transformation'][countK]=nwSqu[countK]
        countK=countK+1
            #Save networks so we don't have to run this every time
            
    accRandom=nwSqu
    return accGrid,ASN300                

In [13]:
nodesList=[[None]*10 for i in range(len(ASN300))]
stimulus=[[[] for i in range(len(ASN300))],[[] for i in range(len(ASN300))]]
init=[]
for i in range(len(ASN300)):
    for j in range(len(ASN300[i])):
        init.append(nonlineartrans,ASN300,onAmp,i,j,nodesList,stimulus)

NameError: name 'ASN300' is not defined

In [ ]:
results=para_run(nonlineartrans,init,nCPU=10)

In [30]:
#Calculate nodesList
nodesList=[[None]*10 for i in range(len(ASN300))]
for i in range(len(ASN300)):
    for j in range(len(ASN300[i])): #for each network:
        if len(ASN300[i][j]['G']) >= 250:
            nodesList[i][j]=[50,100,150,200,250,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)
        elif len(ASN300[i][j]['G']) >= 200 and len(ASN300[i][j]['G']) < 250:
            nodesList[i][j]=[50,100,150,200,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)  
        elif len(ASN300[i][j]['G']) >= 150 and len(ASN300[i][j]['G']) < 200:
            nodesList[i][j]=[50,100,150,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)  
        elif len(ASN300[i][j]['G']) >= 100 and len(ASN300[i][j]['G']) < 150:
            nodesList[i][j]=[50,100,len(ASN300[i][j]['G'])]#range(50, len(ws300[i][0])+1,50)

#### DO NOT DELETE COMMENTS BELOW

In [31]:
# #Current and Communicability Matrices:
# if ( os.path.isfile(r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/simulations_LinearTransformation_29')):
#     print('Running Communicability Analysis')
#     time_index=[[None]*10 for i in range(len(ASN300))]
#     junctions=[[None]*10 for i in range(len(ASN300))]
#     sources=[[None]*10 for i in range(len(ASN300))]
#     drains=[[None]*10 for i in range(len(ASN300))]
#     count1=0
#     count2=0
#     count3=0
#     for i in tqdm(range(len(ASN300))):
#         check_memory()
#         #load corresponding results_ASN file
#         print('Loading Parameter ' + str(i+1) +' - All Networks')
#         [results_ASN]=decompress_pickle(r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/simulations_LinearTransformation_'+str(i))
#         print('Loaded Parameter ' + str(i+1) +' - All Networks')
#         check_memory()
#         for j in range(len(results_ASN)):
#             print('Calculating COMM Matrix for Parameter ' + str(i+1) +', Network ' + str(j+1))
#             nodesListASN_LT,commuMatASN_LT,currMatASN_LT,currGraphASN_LT,time_index[i][j]=commCurr(results_ASN[j]) #calculate communicability every 500 time steps for each network
#             junctions[i][j]=results_ASN[j].junctionSwitch #save junction switch
#             sources[i][j]=results_ASN[j].sources[0]
#             drains[i][j]=results_ASN[j].drains[0]
#               ASN300[i][j]['Graph Theory']['COMM Mat']=commuMatASN_LT
#               ASN300[i][j]['Graph Theory']['Nodes List']=nodesListASN_LT
#               ASN300[i][j][j]['Graph Theory']['Current Matrix']=currMatASN_LT
#               ASN300[i][j][j]['Graph Theory']['Subgraph']=currGraphASN_LT


#             if i < len(cluster1):
#                 cluster1[count1][j]['Graph Theory']['COMM Mat']=commuMatASN_LT
#                 cluster1[count1][j]['Graph Theory']['Nodes List']=nodesListASN_LT
#                 cluster1[count1][j]['Graph Theory']['Current Matrix']=currMatASN_LT
#                 cluster1[count1][j]['Graph Theory']['Subgraph']=currGraphASN_LT
#                 if j == len(results_ASN)-1:
#                     count1=count1+1
#             elif i >= len(cluster1) and i < (len(cluster1) + len(cluster2)):
#                 cluster2[count2][j]['Graph Theory']['COMM Mat']=commuMatASN_LT
#                 cluster2[count2][j]['Graph Theory']['Nodes List']=nodesListASN_LT
#                 cluster2[count2][j]['Graph Theory']['Current Matrix']=currMatASN_LT
#                 cluster2[count2][j]['Graph Theory']['Subgraph']=currGraphASN_LT
#                 if j == len(results_ASN)-1: 
#                     count2=count2+1
#             else:
#                 cluster3[count3][j]['Graph Theory']['COMM Mat']=commuMatASN_LT
#                 cluster3[count3][j]['Graph Theory']['Nodes List']=nodesListASN_LT
#                 cluster3[count3][j]['Graph Theory']['Current Matrix']=currMatASN_LT
#                 cluster3[count3][j]['Graph Theory']['Subgraph']=currGraphASN_LT
#                 if j == len(results_ASN)-1:                
#                     count3=count3+1
#         del results_ASN, commuMatASN_LT, nodesListASN_LT, currMatASN_LT, currGraphASN_LT
        
# else:
#     print('Communicability Analysis Loaded')

In [32]:
# #Save Communicability Matrices so we don't have to load it all again:
# #Update pickle file:
# # if ( os.path.isfile(r'C:/Users/61424/Documents/GitHub/CODE/Analysis/Functional Connectivity/Functional Tasks/networks_LinearTransformation.pkl')):
# name='networks_LinearTransformation.pkl'
# with open(name, 'wb') as f:
#     pickle.dump([ASN300,cluster1,cluster2,cluster3,time_index,nodesList], f)

In [33]:
# #network junctions switches:
# fig1=plt.figure(figsize=(15,15))
# ax=[None]*numNetworks
# axBig = fig1.add_subplot(111)
# axBig.set_frame_on(False)
# axBig.set_yticklabels([])
# axBig.set_xticklabels([])
# plt.xlabel('Time',fontsize=30,labelpad=15)
# plt.ylabel('Num On Switches',fontsize=30,labelpad=15)
# count1=0
# count2=0
# count3=0

# for i in range(numNetworks):
#     ax[i]=fig1.add_subplot(numNetworks/4, numNetworks/6, i+1)
#     fig1.subplots_adjust(hspace=.4)
#     if i < len(cluster1):
#         ax[i].set_title('Cluster 1 Network ' + str(count1+1),fontsize=15)
#         count1=count1+1
#     elif i >= len(cluster1) and i < (len(cluster1) + len(cluster2)):
#         ax[i].set_title('Cluster 2 Network ' + str(count2+1),fontsize=15)
#         count2=count2+1
#     else:
#         ax[i].set_title('Cluster 3 Network ' + str(count3+1),fontsize=15)
#         count3=count3+1
#     plt.plot(np.sum(junctions[i][j],1))
#     #votlage as title 

In [34]:
# i=3000

# sim=results_ASN[0]
# subGraph=getOnGraph(sim,i)
# components = [j for j in nx.connected_components(subGraph)] #all connected nodes in subgraph

# currMat = np.zeros((sim.numOfWires,sim.numOfWires))
# edgeList = sim.connectivity.edge_list
# currMat[edgeList[:,0], edgeList[:,1]] = sim.junctionVoltage[i,:]/sim.junctionResistance[i,:] #-1,:
# currMat = currMat + currMat.T
# currGraph = nx.from_numpy_array(currMat)
# subGraph = getOnGraph(sim, this_TimeStamp=i)

# components = [j for j in nx.connected_components(subGraph)] #all connected nodes in subgraph

# max_ind = np.argmax([len(j) for j in nx.connected_components(subGraph)])
# currGraph = nx.subgraph(currGraph, components[max_ind])

# commu = nx.communicability(currGraph)
# commu_Edges=commu
# subSize = len(currGraph)
# nodesList=list(currGraph.nodes)
# commu_Mat = np.array([commu[k][j] for k in nodesList for j in nodesList]).reshape(subSize,subSize)


## C. Elegans

In [35]:
#On Amp as a function of shortestPath
onAmp=[]
shortestPathElegans=[]
temp=getFarthestPairing(Elegans['adj_matrix'])
shortestPathElegans=nx.shortest_path_length(Elegans['G'],temp[0],temp[1])
onAmpElegans=shortestPathElegans/5

In [41]:
#Instantiate Variables       
stimulus_E=[[],[]]
accSqu=[]
#    accTri=[]
#    accSaw=[]
#    accDbl=[]
maxSqu=[]
#    maxTri=[]
#    maxSaw=[]
#    maxDbl=[]
dt = 1e-2
f=0.5
Time=5
# onAmp=2#[1.5,2,4,4,4,6,6,6,6,6,6,6,4,4,4,4,6,4,4,4,4,3,2,1.5]

#Choose Electrode Pattern

stimulus_E[0].append((stimulus__(biasType='AC',onAmp=onAmpElegans,TimeVector=np.arange(0,Time,dt),f=f)))
stimulus_E[1].append((stimulus__(biasType='Drain',TimeVector=np.arange(0,Time,dt)))) #we don't want this drain to be active during training

#Initialise Output Variables
period=[]
TimeVector=[]
voltage=[]
conductance=[]
switches=[]
results=[None]*len([elegansGraph])
# Voltage=[None]*len(ASN300)
# Switches=[None]*len(ASN300)


nwSqu = []*len([elegansGraph])

#Run Simulations
# for i in range(len(ASN300)): #for each network
print('Network C.Elegans')
#Run Simulations
#     results=[]
# Connectivity=connectivity__('700nw_14533junctions.mat')
stimulus2 = [item for item in stimulus_E] #go through each list in the list and find the ith item
#     set_trace()
results=runSim(connectivity__(graph=elegansGraph),stimulus=stimulus2, contactMode='farthest', T = Time, dt = 0.001, onAmp = onAmpElegans, biasType='AC',f=f,junctionMode='tunneling')
#wires_dict=newNetworkTest[chosenNetwork])
results.frequency=f
results.dt=0.001
period=1/f

TimeVector=results.TimeVector
voltage=results.wireVoltage
conductance=results.conductance
switches=results.junctionSwitch

stepNodes=len(elegansGraph)-1 #first use all nodes
sizes2=len(elegansGraph)
nwOutputs = [None]* int(sizes2/stepNodes)

outputNodes2=[]

for k in range(stepNodes,sizes2+1,stepNodes): #stepping up nodes
    np.random.seed(69)

    outputNodes2.append(voltage[:,np.random.choice(len(voltage[0,:]),size=k,replace=False)]) #take all the times (:) for a random j nodes

    nwOutputs=outputNodes2 #Length of nwOutputs is k (list)

target1=[None]*len(nwOutputs)
#     target2=[None]*len(nwOutputs)
#     target3=[None]*len(nwOutputs)
#     target4=[None]*len(nwOutputs)
#     target5=[None]*len(nwOutputs)

for j in range(len(nwOutputs)):
    target1[j] = (onAmpElegans * (-np.sign(TimeVector % period - period/2)))
#         target2[j] = (4*onAmp[i]/period * abs((TimeVector-period/4) % period - period/2) - onAmp[i])
#         target3[j] = (onAmp[i]/period * (TimeVector % period))
#         target4[j] = (onAmp[i]*np.sin(4*np.pi*(1/period)*TimeVector))


#        nwTri     = np.zeros(len(nwOutputs))
#        nwSaw     = np.zeros(len(nwOutputs))
#        nwDbl     = np.zeros(len(nwOutputs))
#     nwMG      = [[None]*networksLoad,[None]*len(nwOutputs[0])]

ResultSqu=[]
#        ResultTri=[]
#        ResultSaw=[]
accuracy=[]
#        accuracyTri=[]
#        accuracySaw=[]
#        accuracyDbl=[]
output=[]
mSqu=[]
#        mTri=[]
#        mSaw=[]
#        mDbl=[]

# RUN LINEAR TRANSFORMATION
nodesListElegans=[50,100,150,200,250,len(Elegans['G'])]
for j in nodesListElegans: #range(50, len(Elegans['G'])+1,50): #loop through sets of nodes 
    print('Running Regression: ' + str(j) + ' nodes')
    
    ResultSqu=nonLinearTrans(results,'Square',j, repeats=50) #NOKEVregression(target1[j],nwOutputs[j].T)[0]
    
    nwSqu.append(ResultSqu['accuracy'])#(1 - rnMSE)
    
    
#THRESHOLD FOR COMMUNICABILITY MEASURE
results=[results]
# # From Matlab:
# resistance=results.junctionResistance
# for i in range(len(resistance)):
#     if resistance[i]<10000:
#         resistance[i][resistance[i]<10000]=1
# else:
#     resistance[i]=0

Network C.Elegans


2020-06-10 14:08:47,776:INFO:First current path [9, 38, 16, 169, 105, 98, 36] formed at time = 0.773 s.


Running Regression: 50 nodes
Running Regression: 100 nodes
Running Regression: 150 nodes
Running Regression: 200 nodes
Running Regression: 250 nodes
Running Regression: 277 nodes


In [42]:
if (not os.path.isfile('elegans_LinearTransformation.pkl')): #if we haven't saved the file
    #Current and Communicability Matrices
    commuMatElegans_LT=[None]*len(results)
    currMatElegans_LT=[None]*len(results)
    # for i in tqdm(range(len(results))):
    nodesListElegans_LT,commuMatElegans_LT,currMatElegans_LT,new_currGraphElegans_LT,time_indexElegans=commCurr(results[0])

    Elegans['Graph Theory']['COMM Mat']=commuMatElegans_LT
    Elegans['Graph Theory']['Nodes List']=nodesListElegans_LT
    Elegans['Graph Theory']['Current Matrix']=currMatElegans_LT
    Elegans['Graph Theory']['Subgraph']=new_currGraphElegans_LT
    Elegans['Accuracy']['Linear Transformation']=nwSqu
    
    #Save networks so we don't have to run this every time
    print('Saving Elegans Networks')
    name='elegans_LinearTransformation.pkl'
    with open(name, 'wb') as f:
        pickle.dump([Elegans], f)
else:
    name='elegans_LinearTransformation.pkl'
    print('Loading Elegans Networks')
    file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
    [Elegans] = pickle.load(file)

    print('Loaded')

Loading Elegans Networks
Loaded


## Functional Connectivity

### COMMUNICABILITY 12/05 - Need to fix

In [ ]:
#Assign weights to edges based on COMMUNICABILITY:

#Cluster 1:
G_C1=[]
weights_C1=[]
for i in range(len(cluster1)): #for each network
    G_C1.append([None]*len(cluster1[i]['Graph Theory']['COMM Mat']))
    weights_C1.append([None]*len(cluster1[i]['Graph Theory']['COMM Mat']))
    for j in range(len(cluster1[i]['Graph Theory']['COMM Mat'])): #for each timestamp 
        G_C1[i][j]=cluster1[i]['Graph Theory']['Subgraph'][j]
        weights_C1[i][j]=cluster1[i]['Graph Theory']['COMM Mat'][j]
        for e in G_C1[i][j].edges(): #for each edge pairing
            G_C1[i][j][e[0]][e[1]]['weight'] = weights_C1[i][j][e]
            
#Cluster 2:
G_C2=[]
weights_C2=[]
for i in range(len(cluster2)): #for each network
    G_C2.append([None]*len(cluster2[i]['Graph Theory']['COMM Mat']))
    weights_C2.append([None]*len(cluster2[i]['Graph Theory']['COMM Mat']))
    for j in range(len(cluster2[i]['Graph Theory']['COMM Mat'])): #for each timestamp 
        G_C2[i][j]=cluster2[i]['Graph Theory']['Subgraph'][j]
        weights_C2[i][j]=cluster2[i]['Graph Theory']['COMM Mat'][j]
        for e in G_C2[i][j].edges(): #for each edge pairing
            G_C2[i][j][e[0]][e[1]]['weight'] = weights_C2[i][j][e]
            
#Cluster 3:
G_C3=[]
weights_C3=[]
for i in range(len(cluster3)): #for each network
    G_C3.append([None]*len(cluster3[i]['Graph Theory']['COMM Mat']))
    weights_C3.append([None]*len(cluster3[i]['Graph Theory']['COMM Mat']))
    for j in range(len(cluster3[i]['Graph Theory']['COMM Mat'])): #for each timestamp 
        G_C3[i][j]=cluster3[i]['Graph Theory']['Subgraph'][j]
        weights_C3[i][j]=cluster3[i]['Graph Theory']['COMM Mat'][j]
        for e in G_C3[i][j].edges(): #for each edge pairing
            G_C3[i][j][e[0]][e[1]]['weight'] = weights_C3[i][j][e]
            
#C Elegans:
G_E=[]
weights_E=[]
temp=[None]
for i in range(len(temp)): #for each network
    temp[i]=Elegans
    G_E.append([None]*len(temp[i]['Graph Theory']['COMM Mat']))
    weights_E.append([None]*len(temp[i]['Graph Theory']['COMM Mat']))
    for j in range(len(temp[i]['Graph Theory']['COMM Mat'])): #for each timestamp 
        G_E[i][j]=temp[i]['Graph Theory']['Subgraph'][j]
        weights_E[i][j]=temp[i]['Graph Theory']['COMM Mat'][j]
        for e in G_E[i][j].edges(): #for each edge pairing
            G_E[i][j][e[0]][e[1]]['weight'] = weights_E[i][j][e]

In [ ]:
#Plot timeseries:
fig=plt.figure()
plt.plot(stimulus[0][0].signal)#Stimulus[electrode type][network number]
plt.xticks(range(50,151,50),labels=range(500,1501,500))
plt.axvline(50,linestyle='--',color='k')
plt.axvline(150,linestyle='--',color='k')

# PLOT SUBGRAPH ONLY:
fig1=plt.figure(figsize=(15,20))
# for i in range(len(cluster1)):
ax=[None]*(len(cluster1[0]['Graph Theory']['COMM Mat']))

#Find Maximum COMM
vmax1=[]
for i in range(len(G[0])):
    if bool(nx.get_edge_attributes(G[0][i],'weight')):
        edges,weights2 = zip(*nx.get_edge_attributes(G[0][i],'weight').items())
        vmax1.append(max(weights2))
    else:
        weights2=[]
        edges=[]
    
count1=0
times=range(time_index[0][0],time_index[0][1],time_index[0][2])
for j in range(len(cluster1[0]['Graph Theory']['COMM Mat'])): #for each timestamp
    ax[j]=fig1.add_subplot(7,3, j+1)
    ax[j].set_title('Cluster 1, NW 1, Timestamp ' + str(times[j]),fontsize=15)
    if bool(nx.get_edge_attributes(G[0][j],'weight')):
        edges,weights2 = zip(*nx.get_edge_attributes(G[0][j],'weight').items())
    else:
        weights2=[]
        edges=[]
        
    posSub=nx.drawing.layout.kamada_kawai_layout(G[0][j])
    # fig=plt.figure(figsize=(30,30))
    cmap=plt.cm.autumn_r
    vmin = 1
    vmax=max(vmax1)
    
    #NORMALIZE WEIGHTS:
    normalize=plt.Normalize(vmin = vmin, vmax=vmax)
    colors=[cmap(normalize(value)) for value in weights2]
    
    nx.draw(G[0][j],posSub,node_color='k',edgelist=edges,edge_color=colors, width=2,edge_cmap=cmap)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = vmin, vmax=vmax))
    sm._A = []

    plt.colorbar(sm)
    
    count1=count1+1

In [ ]:
# #Plot timeseries:
# fig=plt.figure()
# plt.plot(stimulus[0][0].signal)#Stimulus[electrode type][network number]
# plt.xticks(range(50,151,50),labels=range(500,1501,500))
# plt.axvline(50,linestyle='--',color='k')
# plt.axvline(150,linestyle='--',color='k')
# # plt.xticks(range(0,501,50),labels=range(0,5001,500))

G=[]
clusterNum=3#'E' #CHANGE CLUSTER TO PLOT HERE, for elegans = 'E'
if clusterNum==1:
    G=G_C1
    cluster=cluster1
    accuracy=[[None]*len(cluster1accuracy) for i in range(len(cluster1accuracy[0]))]
    for i in range(len(cluster1accuracy)):
        for j in range(len(cluster1accuracy[i])):
            accuracy[j][i]=cluster1accuracy[i][j]
    count=0
elif clusterNum==2:
    G=G_C2
    cluster=cluster2
    accuracy=[[None]*len(cluster2accuracy) for i in range(len(cluster2accuracy[0]))]
    for i in range(len(cluster2accuracy)):
        for j in range(len(cluster2accuracy[i])):
            accuracy[j][i]=cluster2accuracy[i][j]
    count=len(cluster1)
elif clusterNum==3:
    G=G_C3
    cluster=cluster3
    accuracy=[[None]*len(cluster3accuracy) for i in range(len(cluster3accuracy[0]))]
    for i in range(len(cluster3accuracy)):
        for j in range(len(cluster3accuracy[i])):
            accuracy[j][i]=cluster3accuracy[i][j]
    count=len(cluster1)+len(cluster2)
elif clusterNum=='E':
    G=G_E
    cluster=[None]
    cluster[0]=Elegans
    accuracy=Elegans['Accuracy']['Linear Transformation']

    #Plot Figures
fig=[None]*len(G)
for i in tqdm(range(len(G))): #For Each Network
    #initialise variables:
    pos=[]
    sm=[]
    ax=[None]*(len(cluster[i]['Graph Theory']['COMM Mat']))
    
    #Find Maximum COMM
    vmax1=[]
    for j in range(len(G[i])):
        if bool(nx.get_edge_attributes(G[i][j],'weight')):
            edges,weights2 = zip(*nx.get_edge_attributes(G[i][j],'weight').items())
            vmax1.append(max(weights2))
        else:
            weights2=[]
            edges=[]

    #PLOT NETWORK + COMMUNICABILITY OVERLAYED
    fig[i]=plt.figure(figsize=(40,30))
    pos=nx.drawing.layout.kamada_kawai_layout(cluster[i]['G'])
    for j in range(len(cluster[i]['Graph Theory']['COMM Mat'])): #for each timestamp
        colors=[]
        ax[j]=fig[i].add_subplot(4,5, j+1) #CHANGE THIS FOR MORE/LESS TIMESTAMPS (rows, columns, subplot number)
        ax[j].set_title('Cluster ' + str(clusterNum)+ ', NW ' + str(i+1) + ' Timestamp ' + str(times[j]),fontsize=20)
        nx.draw_networkx(cluster[i]['G'],pos,edge_color='gray',alpha=0.1,with_labels=False,node_size=30)
        colors=range(20)
        cmap=plt.cm.autumn_r
        vmin = 1
        vmax=max(vmax1)

        #NORMALIZE WEIGHTS:
        normalize=plt.Normalize(vmin = vmin, vmax=vmax)
        colors=[cmap(normalize(value)) for value in weights2]
        if clusterNum=='E':
            nx.draw_networkx(cluster[i]['G'],pos,nodelist=[results[i].sources[0]],node_color='g',edgelist=[],with_labels=False)
            nx.draw_networkx(cluster[i]['G'],pos,nodelist=[results[i].drains[0]],node_color='r',edgelist=[],with_labels=False)
        else:
            nx.draw_networkx(cluster[i]['G'],pos,nodelist=[sources[count+i]],node_color='g',edgelist=[],with_labels=False)
            nx.draw_networkx(cluster[i]['G'],pos,nodelist=[drains[count+i]],node_color='r',edgelist=[],with_labels=False)
        nx.draw_networkx_edges(cluster[i]['G'],pos,edgelist=G[i][j].edges,edge_color=colors,alpha=0.8, width=3,edge_cmap=cmap)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = vmin, vmax=vmax))
        sm._A = []

        cb=plt.colorbar(sm)
        cb.set_label(label='Communicability', size='large', weight='bold')
        
        if j == 0:
            if clusterNum == 'E':
                plt.text(0.05,0.9,'Max LT Acc:'+"{0:.2f}".format(np.max(accuracy)),transform=ax[j].transAxes,fontsize=15) #this plots Smallworldness as text in a relative position on each subplot
            else:
                plt.text(0.05,0.9,'Max LT Acc:'+"{0:.2f}".format(np.nanmax(np.array(accuracy[i],dtype=float))),transform=ax[j].transAxes,fontsize=15) #this plots Smallworldness as text in a relative position on each subplot
    if clusterNum=='E':
        plt.savefig(r'C:\Users\61424\Documents\GitHub\CODE\Data\Figures\Functional Connectivity\Communicability\Communicability Conductance Subgraph Elegans.jpg')
    else:
        plt.savefig(r'C:\Users\61424\Documents\GitHub\CODE\Data\Figures\Functional Connectivity\Communicability\Communicability Conductance Subgraph Cluster '+ str(clusterNum) +' NW ' + str(i+1) + '.jpg')